In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Configuration
spark = SparkSession\
    .builder\
    .master('local[2]')\
    .appName('quake_etl')\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
    .getOrCreate() 
#load data
covid = spark.read.csv(r"C:\Users\israt\OneDrive\Desktop\DM_Project\project\covid_history.csv", header=True)

In [2]:
#delete column
delete_columns = ['deathIncrease','inIcuCurrently','states','hospitalizedIncrease','hospitalizedCurrently','negativeIncrease','onVentilatorCurrently','positiveIncrease','totalTestResultsIncrease']
covid_0 = covid.drop(*delete_columns)
covid_0.show(1)
covid_0.printSchema()

+----------+------+---------------+----------------------+--------+----------------------+--------+----------------+
|      date| death|inIcuCumulative|hospitalizedCumulative|negative|onVentilatorCumulative|positive|totalTestResults|
+----------+------+---------------+----------------------+--------+----------------------+--------+----------------+
|2021-03-07|515151|          45475|                776361|74582825|                  4281|28756489|       363825123|
+----------+------+---------------+----------------------+--------+----------------------+--------+----------------+
only showing top 1 row

root
 |-- date: string (nullable = true)
 |-- death: string (nullable = true)
 |-- inIcuCumulative: string (nullable = true)
 |-- hospitalizedCumulative: string (nullable = true)
 |-- negative: string (nullable = true)
 |-- onVentilatorCumulative: string (nullable = true)
 |-- positive: string (nullable = true)
 |-- totalTestResults: string (nullable = true)



In [3]:
covid_1 = covid_0.withColumn('inIcuCumulative',covid_0['inIcuCumulative'].cast(DoubleType())).withColumn('negative',covid_0['negative'].cast(DoubleType())).withColumn('onVentilatorCumulative',covid_0['onVentilatorCumulative'].cast(DoubleType())).withColumn('positive',covid_0['positive'].cast(DoubleType())).withColumn('totalTestResults',covid_0['totalTestResults'].cast(DoubleType())).withColumn('death',covid_0['death'].cast(DoubleType())).withColumn('hospitalizedCumulative',covid_0['hospitalizedCumulative'].cast(DoubleType()))

covid_1.printSchema()

root
 |-- date: string (nullable = true)
 |-- death: double (nullable = true)
 |-- inIcuCumulative: double (nullable = true)
 |-- hospitalizedCumulative: double (nullable = true)
 |-- negative: double (nullable = true)
 |-- onVentilatorCumulative: double (nullable = true)
 |-- positive: double (nullable = true)
 |-- totalTestResults: double (nullable = true)



In [4]:
covid_2=covid_1.withColumn('year', year(to_timestamp('date','yyyy-MM-dd')))
covid_2.show(1)

+----------+--------+---------------+----------------------+-----------+----------------------+-----------+----------------+----+
|      date|   death|inIcuCumulative|hospitalizedCumulative|   negative|onVentilatorCumulative|   positive|totalTestResults|year|
+----------+--------+---------------+----------------------+-----------+----------------------+-----------+----------------+----+
|2021-03-07|515151.0|        45475.0|              776361.0|7.4582825E7|                4281.0|2.8756489E7|    3.63825123E8|2021|
+----------+--------+---------------+----------------------+-----------+----------------------+-----------+----------------+----+
only showing top 1 row



In [6]:
# connect with mongodb by creating database

covid_2.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/covid.covid_2').save()